In [4]:
import polars as pl

df = pl.read_csv('https://raw.githubusercontent.com/ladydisdain5911/python_course_repo/refs/heads/main/train.csv')

print("Типы данных:")
print(df.dtypes)
print("\nКоличество пропусков:")
print(df.null_count())
print("\nОписательная статистика:")
print(df.describe())

print("\nКоличество пассажиров по классам:")
print(df['Pclass'].value_counts())

print("\nКоличество выживших по полу:")
print(df.group_by('Sex').agg(pl.col('Survived').sum()))

print("\nПассажиры старше 44 лет:")
print(df.filter(pl.col('Age') > 44))

Типы данных:
[Int64, Int64, Int64, String, String, Float64, Int64, Int64, String, Float64, String, String]

Количество пропусков:
shape: (1, 12)
┌─────────────┬──────────┬────────┬──────┬───┬────────┬──────┬───────┬──────────┐
│ PassengerId ┆ Survived ┆ Pclass ┆ Name ┆ … ┆ Ticket ┆ Fare ┆ Cabin ┆ Embarked │
│ ---         ┆ ---      ┆ ---    ┆ ---  ┆   ┆ ---    ┆ ---  ┆ ---   ┆ ---      │
│ u32         ┆ u32      ┆ u32    ┆ u32  ┆   ┆ u32    ┆ u32  ┆ u32   ┆ u32      │
╞═════════════╪══════════╪════════╪══════╪═══╪════════╪══════╪═══════╪══════════╡
│ 0           ┆ 0        ┆ 0      ┆ 0    ┆ … ┆ 0      ┆ 0    ┆ 687   ┆ 2        │
└─────────────┴──────────┴────────┴──────┴───┴────────┴──────┴───────┴──────────┘

Описательная статистика:
shape: (9, 13)
┌────────────┬─────────────┬──────────┬──────────┬───┬───────────┬───────────┬───────┬──────────┐
│ statistic  ┆ PassengerId ┆ Survived ┆ Pclass   ┆ … ┆ Ticket    ┆ Fare      ┆ Cabin ┆ Embarked │
│ ---        ┆ ---         ┆ ---      ┆ --- 

In [6]:
import pandas as pd
import bottleneck as bn

df_pd = pd.read_csv('https://raw.githubusercontent.com/ladydisdain5911/python_course_repo/refs/heads/main/train.csv')

mean_age = bn.nanmean(df_pd['Age'].values)
std_age = bn.nanstd(df_pd['Age'].values)
print(f"\nСредний возраст: {mean_age:.2f}, STD: {std_age:.2f}")

# Способ 1: itertuples
df_pd['Fare_new'] = [row.Fare * 1.3 for row in df_pd.itertuples()]

# Способ 2: apply
df_pd['Fare_new'] = df_pd['Fare'].apply(lambda x: x * 1.3)

# Способ 3: векторные операции
df_pd['Fare_new'] = df_pd['Fare'] * 1.3


Средний возраст: 29.70, STD: 14.52


In [13]:
housing = pd.read_csv('https://raw.githubusercontent.com/ladydisdain5911/python_course_repo/refs/heads/main/Housing.csv')

print("\nИсходное использование памяти в байтах:")
print(housing.memory_usage(deep=True))

# price - большие целые значения, int32
# area - int32
# bedrooms, bathrooms, stories - малые значения, хватит int8
# mainroad, guestroom, basement, hotwaterheating, airconditioning, parking, prefarea - boolean
# furnishingstatus - category

optimized = housing.copy()
optimized['price'] = optimized['price'].astype('int32')
optimized['area'] = optimized['area'].astype('int32')
for col in ['bedrooms', 'bathrooms', 'stories', 'parking']:
    optimized[col] = optimized[col].astype('int8')
for col in ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']:
    optimized[col] = optimized[col].map({'yes': True, 'no': False})
optimized['furnishingstatus'] = optimized['furnishingstatus'].astype('category')

print("\nОптимизированное использование памяти в байтах:")
print(optimized.memory_usage(deep=True))

# Сравнение памяти
original_mem = housing.memory_usage(deep=True).sum() / 1024
optimized_mem = optimized.memory_usage(deep=True).sum() / 1024
print(f"\nЭкономия памяти: {original_mem:.2f} KB -> {optimized_mem:.2f} KB")


Исходное использование памяти в байтах:
Index                 132
price                4360
area                 4360
bedrooms             4360
bathrooms            4360
stories              4360
mainroad            32623
guestroom           32252
basement            32346
hotwaterheating     32180
airconditioning     32327
parking              4360
prefarea            32283
furnishingstatus    37461
dtype: int64

Оптимизированное использование памяти в байтах:
Index                132
price               2180
area                2180
bedrooms             545
bathrooms            545
stories              545
mainroad             545
guestroom            545
basement             545
hotwaterheating      545
airconditioning      545
parking              545
prefarea             545
furnishingstatus     858
dtype: int64

Экономия памяти: 251.72 KB -> 10.55 KB
